# **Modelagem molecular por homologia**

Se observarmos a quantidade de sequências depositadas no Genbank e compararmos com o número de sequências de proteínas depositadas no Protein Data Bank, nos deparamos com uma enorme discrepância. Infelizmente, ainda não é possível cristalografar todas as proteínas já sequenciadas por falta de mão de obra, pelo alto custo e por nem sempre a proteína ser passiva de ser cristalografada.

Como alternativa, surge a molelagem molecular por homologia, pois como o nome do método já diz, utiliza de um proteína já cristalografada para modelar e prever a estrutura tridimensional da proteína alvo, desde que a proteína alvo e a proteína molde sejam homólogas.

Ao falar de homologia entre duas sequências (da proteína alvo a ser modelada e do molde - proteína cristalografada), queremos dizer que ambas compartilham de uma estrutura (alfa-helices, beta-pregueadas, regiões de _loop_) semelhante e função bem conservada evolutivamente (ex.: peptidases que tem por função de romper ligações peptídicas).

### Utilizando o _software_ **Modeller** para modelar proteínas

O software Modeller encontra-se na versão 10.4 e pode ser instalado em sistemas operacionais Linux, Windows e Mac. Sua instalação pelo Anaconda Python é muito simples, mas você precisará realizar um cadastro afim de obter a licença do programa.

Para instalar basta copiar e colar as seguintes linhas em seu CMD (Windows) ou  terminal de comando (Linux ou Mac) e posteriormente executá-las, uma por vez:

```conda config --add canans salilab```

```conda install modeller```

Lembre que durante a instalação você será solicitado para digitar a chave do produto (_Product Key_).

### Antes de praticar

Inicialmente precisamos de uma sequência da proteína alvo para prever ou elucidar a sua estrutura tridimensional. Aqui neste _cookbook_, continuaremos a usar como exemplo a sequência da proteína XPRT (Xantina Fosforribosil Transferase) de _Leishmania donovani_ (ID Genbank: AF170105.1). Esta proteína ainda não possui um modelo cristalográfico depositado no Protein Data Bank (PDB), porém possui proteínas homólogas e que foram depositadas no PDB.

A sequência de XPRT encontra-se no arquivo _query_2.fasta_, mas o Modeller não aceita este tipo de arquivo, sendo necessário usar um arquivo em outro formato. Assim, essa sequência será guardada em um arquivo muito semelhante ao fasta, mas no formato _.ali_.

A sequência de XPRT está guardada no arquivo _query_2.ali_. Ao observar a estrutura do arquivo _.ali_, iremos perceber na primeira linha a presença de um sinal de maior que (>) indicando o cabeçalho do arquivo, na qual contém o ID e o nome da proteína abreviado.

Em seguinda, já na segunda linha, observamos novamente o nome da nossa sequência seguida por pontos e valores iguais a 0,00. O que nos indica que esta sequência não possui estrutura cristalográfica conhecida (afinal, o intuito aqui é elucidar esssa estrutura).

Por fim, da terceira linha em diante tem a sequência de aminoácidos da proteína XPRT e ao final dela observamos a presença de um asterísco (_*_) indicando o fim ou o códon de parada.

```
> AF170105.1; XpRT
sequence:XpRT:::::::0.00: 0.00
MLPTHSCKGFVDAQGRVFVDGREYPMASGIVATEDVIQTNIKAMAHTIAKDYKSLSHRDARLSPSTAETA
EAAEAAEAPISYDNPLIIISVLKGSYIFTSDFIRYLGDCGLPHVVDFVRLASYNSGTKSTGQISMLAGLR
FENLRGKHVLIVEDVCDSGRTLRFLRDYIMEKFQPKSIKTLVMVNKEQAARKVDFDPEYFCLAGPNKYIV
GYGFEVNDRYRDLRHILILRDGEATRYPAKL*
```